In [4]:
import pickle
import matplotlib.pyplot as plt
from tools.math import bic_scorer
import os

In [5]:
folder="./cluster/2016-06-18_17.57.34/"

In [6]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(folder) if (isfile(join(folder, f)) and f.startswith("res_D"))]
uniq_simus = set([f.split(".")[0] for f in onlyfiles])
dims = list(set([int(a.split("D")[1].split("K")[0]) for a in uniq_simus]))
dims.sort()
size = list(set([int(a.split("N")[1]) for a in uniq_simus]))
size.sort()
real_clus = list(set([int(a.split("K")[1].split("N")[0]) for a in uniq_simus]))
real_clus.sort()

In [8]:
import numpy as np
res = {}

for d in dims:
    for k in real_clus:
        res[(d,k)]={"em":[],"algo":[]}
        for s in size:
            files = [f for f in listdir(folder) if (isfile(join(folder, f)) and f.startswith("res_D"+str(d)+"K"+str(k)+"N"+str(s)))]
            r_em = np.array([pickle.load(open(folder+f))["gmm"]["score_weights"] for f in files])
            r_algo = np.array([pickle.load(open(folder+f))["sqrt"]["score_weights"] for f in files])
            if len(r_em)!=0 and len(r_algo)!=0:
                res[(d,k)]["em"].append((s, np.median(r_em)))
                res[(d,k)]["algo"].append((s, np.median(r_algo)))

In [9]:
for i in dims:
    for j in real_clus:
        param = (i,j)
        x,algo = zip(*(res[param]["algo"]))
        _,em = zip(*(res[param]["em"]))
        plt.plot(x, algo, 'g-', x, em, 'r--')
        plt.yscale('log')
        plt.xscale('log')
        plt.xlabel("Sample size")
        plt.title('Dim:'+str(param[0])+" K:"+str(param[1]))
        os.makedirs(folder+"results")
        plt.savefig(folder+"results/"+"D"+str(param[0])+"K"+str(param[1]))
        plt.close()

In [10]:
res

{(2,
  3): {'algo': [(1000, 0.00022654704595905843),
   (2000, 0.00017464932986695374)], 'em': [(1000, 0.00021608104711289883),
   (2000, 0.00024705952362168957)]}}